In [1]:
 import boto3
 import json
 from datetime import datetime

 bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')  

 MODEL_ID = "amazon.nova-micro-v1:0"

In [4]:
system_prompt = """
 You are an assistant that summarizes music reviews for a record company.
 Here are examples:

 Review: The latest album by The New Wave Band is a masterpiece! Every track is a hit.
 Summary: Reviewer praises the latest album as a masterpiece with hit tracks.

 Review: I was disappointed with the new single; it lacked the energy of their previous work.
 Summary: Reviewer expresses disappointment, noting a lack of energy compared to previous work.
 """
user_input_review = "This EP is a solid effort with a few standout songs, though some tracks feel repetitive."

In [6]:
no_cache_payload = {
 "system": [
     {"text": system_prompt}
 ],
 "messages": [
     {
         "role": "user",
         "content": [
             {"text": user_input_review + "\nSummary:"}
         ]
     }
 ]
}

no_cache_response = bedrock.converse(
 modelId=MODEL_ID,
 system=no_cache_payload["system"],
 messages=no_cache_payload["messages"]
)

no_cache_output = no_cache_response['output']['message']['content'][0]['text']
no_cache_input_tokens = no_cache_response['usage']['inputTokens']
no_cache_output_tokens = no_cache_response['usage']['outputTokens']
no_cache_tokens = no_cache_input_tokens + no_cache_output_tokens

print("[No Caching] Generated Summary:")
print(no_cache_output)
print(f"Total Tokens Used (No Cache): {no_cache_tokens}")


[No Caching] Generated Summary:
Reviewer acknowledges the EP as a solid effort, highlighting a few standout songs while noting that some tracks feel repetitive.
Total Tokens Used (No Cache): 128


In [7]:
cache_point = {"cachePoint": {"type": "default"}}

payload_with_caching = {
 "system": [
     {"text": system_prompt},
     cache_point
 ],
 "messages": [
     {
         "role": "user",
         "content": [
             {"text": user_input_review + "\nSummary:"}
         ]
     }
 ]
}

response = bedrock.converse(
 modelId=MODEL_ID,
 system=payload_with_caching["system"],
 messages=payload_with_caching["messages"]
)

cache_output = response['output']['message']['content'][0]['text']
cache_input_tokens = response['usage']['inputTokens']
cache_output_tokens = response['usage']['outputTokens']
cache_tokens = cache_input_tokens + cache_output_tokens

print("[With Caching] Generated Summary:")
print(cache_output)
print(f"Total Tokens Used: {cache_tokens}")

[With Caching] Generated Summary:
Reviewer acknowledges the EP as a solid effort, highlighting a few standout songs while noting that some tracks feel repetitive.
Total Tokens Used: 44
